# Fine-tuning a model on the YNAT Task(v1.0)
- [Text Classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=71pt6N0eIrJo)을 KLUE의 주제 분류 데이터셋 YNAT에 맞게 수정 및 번역한 자료입니다.  
  

- **다루는 내용은 다음과 같습니다.**
    - HuggingFace Datasets을 활용하여 KLUE 데이터셋 쉽게 전처리하기
    - HuggingFace Hub에서 사전학습된 언어 모델을 다운로드 받아 사용하고, 학습한 모델을 업로드하여 공유하기
    - `Trainer` 객체를 사용하여 모델 학습 및 평가 & hyperparameter search하기
    - [Weights & Biases](https://wandb.ai/)를 활용하여 실험 관리하기  


# Install packages

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
!pip install datasets wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00


학습한 모델을 커뮤니티에서 공유하거나, 다른 pretrained model 처럼 쉽게 불러와 사용하고 싶다면
1. Hugging Face 웹사이트로부터 얻은 자신의 authentication token을 저장해야 합니다.(회원 가입이 필요하다면 [여기](https://huggingface.co/join)로)
2. 그 다음 아래 셀들의 주석을 풀어서 아이디와 비밀번호를 입력해주세요

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

그 다음 Git-LFS를 설치하고 Git을 설정해야 합니다. 아래 주석을 풀고 e-mail과 이름을 설정해주세요

In [ ]:
!pip install hf-lfs
!git config --global user.email "endnjs33@gmail.com"
!git config --global user.name "DooWonLee"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.1 MB/s eta 0:00:00


```python
# example
# !pip install hf-lfs
# !git config --global user.email "eliza.dukim@gmail.com"
# !git config --global user.name "DaeungKim"
```

이 노트북은 Transformers 4.9.1에서 작성되었습니다.

In [ ]:
import transformers
print(transformers.__version__)

4.37.0


In [ ]:
import accelerate
print(accelerate.__version__)

0.26.1


In [ ]:
# argment setting
task = "ynat"
model_checkpoint = "klue/roberta-base"
batch_size = 512

In [ ]:
from datasets import load_dataset
import pandas as pd

# 데이터셋 로드
dataset = load_dataset('klue', 'ynat')  # 'ynat'는 예시로 사용된 task 이름

# 특정 스플릿(예: 'train')을 데이터프레임으로 변환
df = pd.DataFrame(dataset['train'])

# 결측치 확인
print(df.isna().sum())


guid     0
title    0
label    0
url      0
date     0
dtype: int64


# 1. Loading the Dataset
[🤗 Datasets](https://github.com/huggingface/datasets) 라이브러리를 사용해 데이터셋을 다운로드 합니다. `load_dataset`을 이용하면 쉽게 다운로드 할 수 있습니다.

In [ ]:
from datasets import load_dataset
dataset = load_dataset('klue', task)

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

샘플 하나를 확인해보죠.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [ ]:
dataset['train'][0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

데이터셋이 어떻게 생겼는지 한 눈에 확인해봅시다. 아래함수는 데이터셋에서 임의의 샘플을 추출해 확인합니다.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
import datasets
import pandas as pd
from IPython.display import display, HTML

def show_first_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    df = pd.DataFrame(dataset[:num_examples])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [ ]:
show_first_elements(dataset["train"])

,guid,title,label,url,date
0,ynat-v1_train_00000,유튜브 내달 2일까지 크리에이터 지원 공간 운영,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947,2016.06.30. 오전 10:36
1,ynat-v1_train_00001,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0008384783,2016.05.08. 오전 5:25
2,ynat-v1_train_00002,내년부터 국가RD 평가 때 논문건수는 반영 않는다,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0008254585,2016.03.15. 오후 12:00
3,ynat-v1_train_00003,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0009070646,2017.02.28. 오전 9:54
4,ynat-v1_train_00004,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0009999529,2018.04.03. 오전 7:05
5,ynat-v1_train_00005,야외서 생방송 하세요…액션캠 전용 요금제 잇따라,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=226&oid=001&aid=0008547867,2016.07.18. 오전 9:46
6,ynat-v1_train_00006,월드컵 태극전사 16강 전초기지 레오강 입성종합,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010126131,2018.06.04 08:15
7,ynat-v1_train_00007,미세먼지 속 출근길,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0009817982,2018.01.17. 오전 10:14
8,ynat-v1_train_00008,왓츠앱稅 230원에 성난 레바논 민심…총리사퇴로 이어져종합2보,세계,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=234&oid=001&aid=0011176999,2019.10.30. 오전 4:17
9,ynat-v1_train_00009,베트남 경제 고성장 지속…2분기 GDP 6.71% 성장,세계,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=231&oid=001&aid=0010921012,2019.06.28. 오후 5:42


In [ ]:
show_random_elements(dataset["train"])

,guid,title,label,url,date
0,ynat-v1_train_16554,홍남기 2차 공공기관 이전 검토 중…굉장히 신중한 입장,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010390411,2018.10.10. 오후 2:01
1,ynat-v1_train_22249,언론 브리핑 마친 이효성 방송통신위원장,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0010972002,2019.07.22. 오전 11:29
2,ynat-v1_train_12569,베스트셀러 김영하 여행의 이유 5주째 1위,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0010845261,2019.05.24. 오전 10:00
3,ynat-v1_train_05289,아시안게임 김학범호 인도네시아 도착 금빛 사냥 출발,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010264665,2018.08.12 02:38
4,ynat-v1_train_30291,김동연 내년 자율차 등 8대 선도사업에 5조 이상 투자1보,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=258&oid=001&aid=0010258619,2018.08.08. 오후 3:15
5,ynat-v1_train_17002,흥이 넘치는 요스바니 리시브 부담 없어…범실은 줄여야,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010617201,2019.02.01 21:09
6,ynat-v1_train_38432,구글 AI스피커 구글홈 11일 국내 선보인다,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=226&oid=001&aid=0010320882,2018.09.04. 오후 2:27
7,ynat-v1_train_02235,대학교수들 언론사 대학평가 공정하지 않아…거부해야,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0010105789,2018.05.24. 오후 6:11
8,ynat-v1_train_23124,사우디 아람코 이틀째 주가 하락…시총 2조달러 아래로,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=234&oid=001&aid=0011285630,2019.12.18. 오후 9:52
9,ynat-v1_train_42346,부천 공장 사무실서 불…2천여만원 피해,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=257&oid=001&aid=0011288801,2019.12.20. 오전 10:23


# 2. Processing the data

🤗 Transformers `Tokenizer`는 모델의 입력 텍스트를 토크나이징하고(또한 각 토큰들에 할당된 IDs로 변환하는 작업도 함께 수행), 모델이 입력받는 포맷으로 만들어 줍니다(각 모델이 입력 받아야할 입력 포맷, BERT Pretraining의 경우 attention mask, token type ids 까지 함꼐 처리).

이 모든 작업을 하기 위해서 먼저 `AutoTokenizer.from_pretrained`를 사용해 토크나이저를 준비해야합니다. 이걸 사용하면

- 사용하고자 하는 모델과 관련된 토크나이저를 얻을 수 있습니다.
- 특정 체크포인트를 pretraining할 때 사용된 vocabulary를 다운로드 할 수 있습니다.

vocabulary는 캐시되므로, 다음번에 셀을 실행할 때 다시 다운로드 되지 않습니다.

In [ ]:
model_checkpoint

'klue/roberta-base'

In [ ]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

토크나이저를 이용해 문장을 인코딩할 수 있습니다. 하나의 문장을 입력하거나 문장 쌍을 입력할 수 있습니다

In [ ]:
tokenizer("가짜연구소 3기 KLUE 벤치마크 팀 화이팅!!!", "잘 부탁 드려요")

{'input_ids': [0, 8711, 19648, 23, 2015, 47, 2237, 2309, 2105, 9262, 9112, 1823, 21121, 5, 5, 5, 2, 1521, 5527, 20469, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
print(tokenizer("가짜연구소 3기 KLUE 벤치마크 팀 화이팅!!!").tokens())

In [ ]:
print(tokenizer("가짜연구소 3기 KLUE 벤치마크 팀 화이팅!!!", "잘 부탁 드려요").tokens())

YNAT 태스크에서는 분류 대상 문장이 포함된 열인 'title'을 모델 입력으로 사용합니다.

In [ ]:
print(f"Sentence: {dataset['train'][0]['title']}")

Sentence: 유튜브 내달 2일까지 크리에이터 지원 공간 운영


전처리를 위해 사용할 함수를 작성해봅시다. `tokenizer`의 입력으로 dataset의 'text'열을 입력하면서 `truncation=True`를 설정하면됩니다.
 이 옵션은 모델이 입력 받을 수 있는 토큰 최대 길이를 벗어난 토큰들을 잘라주는 옵션입니다.

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['title'], truncation=True)

이 함수는 입력 데이터를 하나 이상 받을 수 있어 배치로 처리 가능합니다.
이 경우 각각의 key의 value는 리스트 형태로 반환됩니다.

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 10637, 8474, 22, 2210, 2299, 2118, 28940, 3691, 4101, 3792, 2], [0, 24905, 1042, 4795, 19982, 2129, 121, 6904, 16311, 3, 14392, 2], [0, 4172, 3797, 3728, 2107, 2134, 3777, 904, 6022, 2332, 2113, 2259, 4523, 1380, 2259, 2062, 2], [0, 12417, 2155, 7840, 604, 2859, 3873, 11554, 2522, 1539, 2073, 8446, 6626, 18818, 575, 2], [0, 13203, 2179, 2366, 4197, 7551, 2096, 8542, 2088, 2353, 886, 1244, 4393, 2027, 22, 2207, 8189, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

이 함수를 전체 데이터셋의 문장에 적용시키기 위해서, 앞서 생성한 `dataset` 객체의 `map` 메서드를 사용합니다. 이 때 `dataset`의 모든 splits에 적용되어 한 번의 실행 만으로 'train', 'validation' 셋이 모두 모델의 입력으로 변환됩니다.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

변환 결과는 🤗 Datasets 라이브러리에 의해 자동으로 캐시되어 다음 번에 노트북을 실행할 때는 캐시된 데이터를 불러와 시간을 절약합니다. 🤗 Datasets 라이브러리는 `map` 메서드에 입력되는 함수의 수정 여부를 체크하여 캐시된 데이터의 사용여부를 결정합니다. 이때 🤗 Datasets 캐시된 파일을 사용여부를 알려주며, `map` 메서드 호출 시 `load_from_cache_file=False`으로 변환을 강제할 수 있습니다.
  
`batched=True`를 사용해서 배치로 묶어서 텍스트를 인코딩 한 것을 볼 수 있는데, 이건 앞서 로드한 fast tokenizer의 이점을 활용하기 위함입니다. fast tokenizer는 멀티 쓰레딩을 사용하여 텍스트를 배치 형태로 거의 동시에 처리 합니다.

# 3. Fine-tuning the model

데이터가 준비되었으니, Pretrained model을 다운 받아 fine-tune 해봅시다. YNAT은 문장 분류와 관련된 task이므로,  `AutoModelForSequenceClassification` 클래스를 사용합니다. tokenizer 때처럼,  `from_pretrained` 메서드로 모델을 다운로드 받고 캐시합니다.  
딱 한 가지 설정해줘야할 것은 label의 개수입니다(YNAT은 총 7개를 label로 구성됨):

In [ ]:
dataset['train'].features['label']

ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

이 노트북은 [Model Hub](https://huggingface.co/models)에서 Pretraining된 모델이 존재할 경우(`[모델명]ForPreTraining`) 모델 체크포인트를 다운로드하여 YNAT task를 학습합니다.

In [ ]:
model_checkpoint

'klue/roberta-base'

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


위의 경고메시지는 다음을 의미합니다.

- `BertForPreTraining`엔 있지만 `BertForSequenceClassification`엔 없는 레이어는 버리고
- `BertForSequenceClassification` 엔 있지만 `BertForPreTraining`엔 없는 레이어는 랜덤 초기화.

따라서 `BertForSequenceClassification` 모델을 fine-tune 하지 않으면 좋은 성능을 얻지 못할 것이라는 것이죠🙂[(reference)](https://github.com/huggingface/transformers/issues/5421)

HuggingFace 라이브러리에서는 `Trainer` 객체를 사용하여 복잡한 학습 코드를 짤 필요 없이 편리하게 학습할 수 있습니다.  
`Trainer` 객체는 `TrainingArgument`를 입력 받는데 모델 학습을 위해 설정하는 값들이 attribute로 들어있는 클래스 입니다. 입력받는 인자 중에 필수적인 것은 `output_dir`인데, 모델의 checkpoint를 저장하기 위한 경로를 의미합니다.

In [ ]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "ynat")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "steps", # 'steps'
    load_best_model_at_end=True,
    logging_steps=50,
    save_steps = 500,


    # Dataset
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=200,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation
    metric_for_best_model='eval_f1',
    evaluation_strategy = "steps",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

학습을 끝까지 수행한 모델이 최고 성능을 내는 모델은 아닙니다. 따라서 학습이 종료되었을 때 `Trainer` 객체가 `metric_name`에서 설정한 `metric`에 따라 가장 좋은 성능을 보인 모델을 불러오도록 `metric_for_best_model` 파라미터를 설정합니다.  
주석처리한 마지막 두 파라미터는 학습이 끝날 때 모델을 [Hub](https://huggingface.co/models)에 push 할 때 설정합니다.

마지막으로 `Trainer` 세팅을 위해서 필요한 것은 metric을 계산하는 방법을 정의해줍니다. 먼저 `datasets` 라이브러리에서 F1-score metric을 로드하여 `meric_macrof1` 변수에 할당한 뒤, 모델의 출력값을 입력으로 받는 `compute_metrics` 함수를 정의하여 `meric_macrof1`을 계산한 결과를 출력합니다. 입력에 적절하도록 모델의 출력값을 전처리 해주는 과정이 포함됩니다.

In [ ]:
# datasets 라이브러리에서 제공하는 Evaluation metric의 리스트를 확인합니다.
from datasets import list_metrics, load_metric
metrics_list = list_metrics()
len(metrics_list)
print(', '.join(metric for metric in metrics_list))

<ipython-input-27-e741eeb3c60a>:3: FutureWarning: list_metrics is deprecated and will be removed in the next major version of datasets. Use 'evaluate.list_evaluation_modules' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metrics_list = list_metrics()


accuracy, bertscore, bleu, bleurt, brier_score, cer, character, charcut_mt, chrf, code_eval, comet, competition_math, confusion_matrix, coval, cuad, exact_match, f1, frugalscore, glue, google_bleu, indic_glue, mae, mahalanobis, mape, mase, matthews_correlation, mauve, mean_iou, meteor, mse, nist_mt, pearsonr, perplexity, poseval, precision, r_squared, recall, rl_reliability, roc_auc, rouge, sacrebleu, sari, seqeval, smape, spearmanr, squad, squad_v2, super_glue, ter, trec_eval, wer, wiki_split, xnli, xtreme_s, AlhitawiMohammed22/CER_Hu-Evaluation-Metrics, BucketHeadP65/confusion_matrix, BucketHeadP65/roc_curve, DaliaCaRo/accents_unplugged_eval, DarrenChensformer/eval_keyphrase, DarrenChensformer/relation_extraction, Drunper/metrica_tesi, Felipehonorato/eer, Fritz02/execution_accuracy, GMFTBY/dailydialog_evaluate, GMFTBY/dailydialogevaluate, He-Xingwei/sari_metric, Ikala-allen/relation_extraction, JP-SystemsX/nDCG, Josh98/nl2bash_m, KevinSpaghetti/accuracyk, LottieW/accents_unplugged_ev

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00


In [ ]:
import evaluate

# 평가 지표 리스트를 로드합니다.
metrics_list = evaluate.list_evaluation_modules()

# 리스트의 길이와 함께 평가 지표를 출력합니다.
print(len(metrics_list))
print(', '.join(metric for metric in metrics_list))


138
lvwerra/test, jordyvl/ece, angelina-wang/directional_bias_amplification, cpllab/syntaxgym, lvwerra/bary_score, hack/test_metric, yzha/ctc_eval, codeparrot/apps_metric, mfumanelli/geometric_mean, daiyizheng/valid, erntkn/dice_coefficient, mgfrantz/roc_auc_macro, Vlasta/pr_auc, gorkaartola/metric_for_tp_fp_samples, idsedykh/metric, idsedykh/codebleu2, idsedykh/codebleu, idsedykh/megaglue, cakiki/ndcg, Vertaix/vendiscore, GMFTBY/dailydialogevaluate, GMFTBY/dailydialog_evaluate, jzm-mailchimp/joshs_second_test_metric, ola13/precision_at_k, yulong-me/yl_metric, abidlabs/mean_iou, abidlabs/mean_iou2, KevinSpaghetti/accuracyk, NimaBoscarino/weat, ronaldahmed/nwentfaithfulness, Viona/infolm, kyokote/my_metric2, kashif/mape, Ochiroo/rouge_mn, giulio98/code_eval_outputs, leslyarun/fbeta_score, giulio98/codebleu, anz2/iliauniiccocrevaluation, zbeloki/m2, xu1998hz/sescore, dvitel/codebleu, NCSOFT/harim_plus, JP-SystemsX/nDCG, sportlosos/sescore, Drunper/metrica_tesi, jpxkqx/peak_signal_to_nois

In [ ]:
metric_macrof1 = evaluate.load('f1')

In [ ]:
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions.argmax(-1)
    labels = eval_pred.label_ids
    return metric_macrof1.compute(predictions=predictions,
                                  references=labels, average='macro')

In [ ]:
encoded_dataset['validation'].info

DatasetInfo(description='', citation='', homepage='', license='', features={'guid': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'label': ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None), 'url': Value(dtype='string', id=None), 'date': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='klue', config_name='ynat', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=10115294, num_examples=45678, shard_lengths=None, dataset_name='klue'), 'validation': SplitInfo(name='validation', num_bytes=2040320, num_examples=9107, shard_lengths=None, dataset_name='klue')}, download_checksums={'hf://dataset

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

앞에서 `tokenizer`를 사용해 전처리를 했음에도 `Trainer`의 입력으로 다시 넣는 이유는 패딩을 적용해서 입력 샘플들을 동일한 길이로 만들기 위해 (데이터 로더의 마지막 과정에서) 사용하기 때문입니다. 모델에 따라 패딩에 대한 기본 설정이 다르기 때문에(왼쪽 패딩, 오른쪽 패딩, 또는 패딩 인덱스 번호 설정 등) `Trainer`는 이와 관련된 작업을 수행할 수 있은 `tokenizer` 객체를 사용합니다. `tokenizer` 대신에 `data_collator`를 커스텀하여 `tokenizer`를 적용하였을 때와 동일한 형태의 입력을 만들어 줄 수도 있습니다.

huggingface는 모델 학습 로그를 기록할때 Tensorboard 또는 [Weights & Biases](https://wandb.ai/site)를 사용할 수 있습니다. 여기서는 Weights & Biases를 사용해 보겠습니다.

In [ ]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

실험 관리를 위해서 id값을 생성합니다.

In [ ]:
id = wandb.util.generate_id()
print(id)

2apt4qib


- project : 실험기록을 관리할 프로젝트 이름. 없을 시 입력받은 이름으로 생성, 여기선 예시로 klue로 설정
- entity : weights & biases 사용자명 또는 팀 이름
- id : 실험에 부여된 고유 아이디
- name : 실험에 부여한 이름
- resume : 실험을 재개할 떄, 실험에 부여한 고유 아이디를 입력

In [ ]:
wandb.init(project='klue',
           entity='wpfkcm33',
           id='2apt4qib',
        #    name='[EXPERIMENT_NAME]',
           resume=True,
            )

# wandb.init(project='klue',
#            entity='dukim',
#            id='319934ii',
#            name='ynat'
#         #    resume=True,
#            )

wandb: WARNING Tried to auto resume run with id h4h44wi9 but id 2apt4qib is set.


eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/f1,0.85039
eval/loss,0.46611
eval/runtime,13.4112
eval/samples_per_second,679.06


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1
50,0.316200,0.406879,0.860990
100,0.276000,0.417583,0.858710
150,0.270600,0.403576,0.863127
200,0.294100,0.423207,0.859022
250,0.277800,0.399437,0.862329
300,0.257500,0.397938,0.862832
350,0.238900,0.400767,0.865222
400,0.225800,0.395022,0.865279
450,0.209700,0.393799,0.867212


TrainOutput(global_step=450, training_loss=0.2629499032762316, metrics={'train_runtime': 1026.7341, 'train_samples_per_second': 222.443, 'train_steps_per_second': 0.438, 'total_flos': 2940227422225800.0, 'train_loss': 0.2629499032762316, 'epoch': 5.0})

`evaluate` 메서드를 사용하여 `Trainer`가 best 모델로 불러온 모델의 성능을 확인해볼 수 있습니다.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.39379921555519104,
 'eval_f1': 0.8672117930190295,
 'eval_runtime': 13.4601,
 'eval_samples_per_second': 676.591,
 'eval_steps_per_second': 1.337,
 'epoch': 5.0}

학습이 끝나면 wandb 를 꺼줍니다

In [ ]:
wandb.finish()

eval/f1,▃▁▅▁▄▄▆▆██
eval/loss,▄▇▃█▂▂▃▁▁▁
eval/runtime,▄▄▃▂▃▃▄█▄▁
eval/samples_per_second,▅▄▅▇▆▆▅▁▅█
eval/steps_per_second,▅▄▅▇▆▆▅▁▅█
train/epoch,▁▁▂▂▃▃▄▄▄▄▅▅▆▆▇▇████
train/global_step,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇████
train/learning_rate,▃▅▆█▇▅▄▂▁
train/loss,█▅▅▇▅▄▃▂▁
train/total_flos,▁
train/train_loss,▁


현재 tokenizer를 비롯한 모델을 Hub에 업로드하려면 `push_to_hub()` 메서드를 사용합니다.

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Doowon96/roberta-base-finetuned-ynat/commit/ab63ecfa48702c751fa098becd84753fa07e0d7c', commit_message='End of training', commit_description='', oid='ab63ecfa48702c751fa098becd84753fa07e0d7c', pr_url=None, pr_revision=None, pr_num=None)

업로드한 모델을 `Hub-user-name/사용자가 지정한 이름`으로 바로 다운로드하여 사용할 수 있습니다.

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 7
# {HuggingFace Model Hub 사용자 아이디}/{push_to_hub_model_id에서 설정한 값}
model = AutoModelForSequenceClassification.from_pretrained('Doowon96/roberta-base-finetuned-ynat', num_labels=num_labels)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

# 4. Hyperparameter search

`Trainer`는 [optuna](https://optuna.org/) 또는 [Ray Tune](https://docs.ray.io/en/latest/tune/)를 이용한 hyperparameter search를 지원합니다. 이 섹션을 실습하기 위해서 두 라이브러리 모두 설치되어 있어야 합니다. 아래 주석을 풀고 실행시켜 라이브러리를 설치한 뒤 다음 셀을 실행시켜 주세요

In [ ]:
! pip install optuna
! pip install ray[tune]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.6 MB/s eta 0:00:00


In [ ]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = os.path.join("test-klue", "ynat")
logging_dir = os.path.join(output_dir, 'logs')
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,
    # overwrite_output_dir=True,

    # Model Save & Load
    save_strategy = "steps", # 'steps'
    load_best_model_at_end=True,
    logging_steps=50,
    save_steps = 500,


    # Dataset
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    # Optimizer
    learning_rate=2e-5, # 5e-5
    weight_decay=0.01,  # 0
    warmup_steps=50,

    # Resularization
    # max_grad_norm = 1.0,
    # label_smoothing_factor=0.1,


    # Evaluation
    metric_for_best_model='eval_f1',
    evaluation_strategy = "steps",

    # HuggingFace Hub Upload
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-finetuned-{task}",

    # Logging
    logging_dir=logging_dir,
    report_to='wandb',

    # Randomness
    seed=42,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1760: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case Doowon96/roberta-base-finetuned-ynat).
  warnings.warn(


hyperparameter search 동안에 `Trainer`는 학습을 여러번 실행합니다. 따라서 모델이 매 학습마다 다시 초기화 될 수 있도록 모델이 함수에 의해 정의되도록 합니다.

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

그 다음, 모델을 직접 입력하는 대신 `model_init`으로 초기화하는 함수를 입력합니다. 나머지는 앞에서 했던 것과 거의 동일하게 `Trainer`를 초기화 해주었습니다.

전체 데이터셋에 대해서 사용할 떄 몇몇 태스크에서는 시간이 오래 걸릴 수 있습니다. 이럴 땐 아래 예시 코드처럼  `dataset` 객체의 `shard` 메서드를 이용해 데이터 셋의 일부(예시에선 1/10)만을 선택하여 hyperparameter를 찾아보는 것도 한 방법이 될 수 있습니다. 찾은 hyperparameter를 전체 데이터셋에 대해 학습할 때 적용하여 최종 모델을 학습 시킵니다.

```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10)
```


In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"].shard(index=1, num_shards=10),
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
id = wandb.util.generate_id()
print(id)

9886492n


- project : 실험기록을 관리할 프로젝트 이름. 없을 시 입력받은 이름으로 생성
- entity : weights & biases 사용자명 또는 팀 이름
- id : 실험에 부여된 고유 아이디
- name : 실험에 부여한 이름
- resume : 실험을 재개할 떄, 실험에 부여한 고유 아이디를 입력

In [ ]:
wandb.init(project='roberta_para1',
           entity='wpfkcm33',
           id='[9886492n]',
        #    name='[EXPERIMENT_NAME]'
            resume=True,
           )

# wandb.init(project='klue-ynat-hps',
#            entity='dukim',
#            id='1stglc5n'
#         #    name='ynat'
#         #    resume=True,
#            )

eval/f1,▁▁▄███████
eval/loss,██▆▂▁▁▁▁▁▁
eval/runtime,██▃▁▂▄▅▄▂▃
eval/samples_per_second,▁▁▆█▇▅▄▅▇▆
eval/steps_per_second,▁▁▆█▇▅▄▅▇▆
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▆▆▇▇▇▇█
train/global_step,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇█
train/learning_rate,▁▃▆██████▇▇
train/loss,██▇▃▂▂▂▂▁▂▂
eval/f1,0.81624
eval/loss,0.69828


`Trinaer` 객체의 `hyperparameter_search` 메서드를 사용해 hyperparameter search를 수행합니다. 입력 파라미터는 다음과 같습니다.
- `hp_space` : hyperparameter search를 수행할 딕셔너리를 반환하는 함수를 입력받습니다. 값을 설정하지 않을 경우 optuna의 기본값을 사용합니다.
 - optuna를 사용할 경우:  
```python
def my_hp_space(trial):
        return {
            "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
            "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
            "seed": trial.suggest_int("seed", 1, 40),
            "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
        }
```
 - ray를 사용할 경우:  
```python
def my_hp_space_ray(trial):
        from ray import tune

        return {
            "learning_rate": tune.loguniform(1e-4, 1e-2),
            "num_train_epochs": tune.choice(range(1, 6)),
            "seed": tune.choice(range(1, 41)),
            "per_device_train_batch_size": tune.choice([4, 8, 16, 32, 64]),
        }
```


- `computive_objective` : 최대화하거나 최소화할 목적함수를 받습니다. 기본값으로 모델의 `evaluate` 메서드에 의해 반환되는 metric값(여기선 F1-score)를 사용합니다.
    ```python
    def my_objective(metrics):
        return metrics["eval_f1"]
    ```
- `n_trials` : 테스트할 실험의 개수를 설정합니다(기본값 100).
- `direction` : `computive_objective`값의 최적화의 방향을 정합니다. `'minimize'`(기본값) 또는 `'maximize'`.

In [ ]:
#겁나 오래걸림! 사용전 logging옵션+콜랩 유지시간+n_trials횟수 확인
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize")

[I 2024-01-23 07:32:01,076] A new study created in memory with name: no-name-c0244336-2477-4b4c-8ee3-ec33e5612eff
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1
50,1.338800,0.687694,0.782654
100,0.600100,0.659538,0.796913
150,0.509000,0.510388,0.831996
200,0.353600,0.527965,0.828760
250,0.360700,0.474561,0.844573


[I 2024-01-23 07:33:58,378] Trial 0 finished with value: 0.8445728233393962 and parameters: {'learning_rate': 8.591620050076783e-05, 'num_train_epochs': 2, 'seed': 16, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.8445728233393962.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/f1,▁▃▇▆█
eval/loss,█▇▂▃▁
eval/runtime,██▂▁▄
eval/samples_per_second,▁▁▇█▅
eval/steps_per_second,▁▁▇█▅
train/epoch,▁▁▂▂▄▄▅▅▇▇█
train/global_step,▁▁▂▂▄▄▅▅▇▇█
train/learning_rate,█▆▅▃▁
train/loss,█▃▂▁▁
train/total_flos,▁
train/train_loss,▁


Step,Training Loss,Validation Loss,F1
50,1.497000,1.089547,0.673421
100,0.566500,0.675825,0.801295
150,0.508300,0.696052,0.789549
200,0.386500,0.608665,0.820191
250,0.385000,0.502578,0.835954
300,0.357000,0.550018,0.829402
350,0.233700,0.585188,0.832439
400,0.213600,0.528506,0.844910
450,0.192600,0.614358,0.831360
500,0.135900,0.705953,0.828427


...

Step,Training Loss,Validation Loss,F1
50,1.934000,2.002475,0.017051
100,1.917400,1.994687,0.167731
150,1.698400,1.637350,0.579749
200,1.258500,1.273312,0.673478
250,0.948700,1.110450,0.681495
300,0.807200,1.060998,0.671731
350,0.754900,0.949426,0.722879
400,0.664400,0.813968,0.771955
450,0.630600,0.784441,0.773831
500,0.609300,0.745084,0.785053


[I 2024-01-23 08:11:17,800] Trial 3 finished with value: 0.8416673610934298 and parameters: {'learning_rate': 3.851823303957499e-06, 'num_train_epochs': 5, 'seed': 30, 'per_device_train_batch_size': 8}. Best is trial 2 with value: 0.8489315109929375.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/f1,▁▂▆▇▇▇▇█████████████████████████████████
eval/loss,██▆▄▄▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▃▇▄▁▂▄▃▄█▆▂▄▃▃▃▇▆▃▃▃▃▄█▃▃▃▃▂▅█▃▃▄▄▅▄█▄▃▅
eval/samples_per_second,▆▂▅█▇▅▆▅▁▃▇▅▅▆▆▂▃▆▆▆▆▅▁▆▆▆▅▆▄▁▆▆▅▅▄▅▁▅▆▄
eval/steps_per_second,▅▂▅█▇▅▆▅▁▃▇▅▅▆▆▂▃▆▆▆▆▅▁▆▆▆▅▆▄▁▆▆▅▅▄▅▁▅▆▄
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,██▇▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


Step,Training Loss,Validation Loss,F1
50,1.936600,2.097200,0.032178
100,1.797800,1.658867,0.509110
150,1.021000,1.123053,0.659201
200,0.730200,1.105814,0.627433
250,0.642900,0.895386,0.734523
300,0.602500,0.877573,0.722191
350,0.641400,0.781848,0.775921
400,0.532500,0.799319,0.773536
450,0.542800,0.740598,0.784078
500,0.668000,0.635532,0.808030


[I 2024-01-23 08:32:30,909] Trial 4 finished with value: 0.8479772386494879 and parameters: {'learning_rate': 9.563310020185664e-06, 'num_train_epochs': 3, 'seed': 19, 'per_device_train_batch_size': 4}. Best is trial 2 with value: 0.8489315109929375.


`hyperparameter_search` 메서드는 `BestRun` 객체를 반환합니다. 이 객체는 최대화된 objective의 값과 이때 선택된 hyperparameter를 포함합니다.

In [ ]:
best_run

BestRun(run_id='2', objective=0.8489315109929375, hyperparameters={'learning_rate': 9.520888692829572e-06, 'num_train_epochs': 4, 'seed': 4, 'per_device_train_batch_size': 8}, run_summary=None)

# Appendix

## 1. optuna를 사용한 hyperparameter search code snippet

```python
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

class MemorySaverCallback(TrainerCallback):
    "A callback that deleted the folder in which checkpoints are saved, to save memory"
    def __init__(self, run_name):
        super(MemorySaverCallback, self).__init__()
        self.run_name = run_name

    def on_train_begin(self, args, state, control, **kwargs):
        print("Removing dirs...")
        if os.path.isdir(f'./{self.run_name}'):
            import shutil
            shutil.rmtree(f'./{self.run_name}')
        else:
            print("\n\nDirectory does not exists")

training_args = TrainingArguments(
    RUN_NAME,
    num_train_epochs=15,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=False,
    overwrite_output_dir=True,
    save_strategy="no",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)

trainer = Trainer(
    model_init=partial(MyNet,2),
    args=training_args,
    train_dataset=training_opos.select(range(2000)),
    eval_dataset=validating_opos,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2), MemorySaverCallback(RUN_NAME)]
)

def my_hp_space_optuna(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 2e-4, log=True),
        "warmup_steps":  trial.suggest_float("warmup_steps", 0., 0.9, step=0.3),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1)
    }
def my_objective(metrics):
    return metrics["eval_f1"]

sa = trainer.hyperparameter_search(
    direction="maximize",
    n_trials=1,
    hp_space=my_hp_space_optuna,
    compute_objective=my_objective
)
```

# Reference
- [Text Classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=71pt6N0eIrJo)
- [HuggingFace Datasets Docs](https://huggingface.co/docs/datasets/index.html)
- [HuggingFace Transformers Docs](https://huggingface.co/transformers/index.html)
- [Using hyperparameter-search in Trainer](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/55)